In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio as rio
import datetime
import earthpy as et
import os
import warnings
warnings.filterwarnings('ignore')

C:\Users\ikey\miniconda3\envs\pyke\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [30]:
etak_14 =pd.read_csv('../grid_elev_etak/etak_grid_14_5.csv')

etak_18 =pd.read_csv('../grid_elev_etak/etak_grid_18_5.csv')

grid_dist = pd.read_csv('../grid_elev_etak/grids_dist_coast.csv')

grid_elev = pd.read_csv('../grid_elev_etak/grids_elev_25.csv')

grids = gpd.read_file('../grid_elev_etak/GRD1000_2017.shp')

ndvi_10_dir  = '../MODIS/NDVI/NDVI_10_AT_3301_16C'

ndvi_14_dir  = '../MODIS/NDVI/NDVI_14_AT_3301_16C'

ndvi_18_dir  = '../MODIS/NDVI/NDVI_18_AT_3301_16C'

lst_10_dir = '../MODIS/LST/HWP_10_AT_D_3301'

lst_14_dir = '../MODIS/LST/HWP_14_AT_D_3301'

lst_18_dir = '../MODIS/LST/HWP_18_AT_D_3301'

Out_folder = '../Tables/'


image_wc = '.tif'

In [6]:
grids

,GRD_INSPIR,area,NEAR_FID,NEAR_DIST,geometry
0,1kmN6465E0369,1000000.0,3,0.000000,"POLYGON ((369000.000 6465000.000, 369000.000 6..."
1,1kmN6466E0369,1000000.0,3,0.000000,"POLYGON ((369000.000 6466000.000, 369000.000 6..."
2,1kmN6467E0369,1000000.0,3,0.000000,"POLYGON ((369000.000 6467000.000, 369000.000 6..."
3,1kmN6468E0369,1000000.0,3,0.000000,"POLYGON ((369000.000 6468000.000, 369000.000 6..."
4,1kmN6465E0370,1000000.0,3,0.000000,"POLYGON ((370000.000 6465000.000, 370000.000 6..."
...,...,...,...,...,...
45663,1kmN6547E0707,1000000.0,3,42907.705297,"POLYGON ((707000.000 6547000.000, 707000.000 6..."
45664,1kmN6548E0707,1000000.0,3,41919.089897,"POLYGON ((707000.000 6548000.000, 707000.000 6..."
45665,1kmN6549E0707,1000000.0,3,40931.027615,"POLYGON ((707000.000 6549000.000, 707000.000 6..."
45666,1kmN6550E0707,1000000.0,3,39943.559500,"POLYGON ((707000.000 6550000.000, 707000.000 6..."


In [7]:
etak_18.head()

,GRD_INSPIR,WATER,ROADS,URBAN_GREENERY,OTHER,BUILT_UP_COMMERCIAL_USE,EXTRACTION_SITES,OPEN_SPACE_COMMERCIAL,CROPLAND,AGRICULTURAL_LAND,NATURAL_VEGETATION,OPEN_SPACE_MIXED_USE,FOREST,WETLANDS,BUILT_UP_MIXED_USE,BUILT_UP_AGRICULTURE
0,1kmN6465E0369,943425,0,0,0,0,0,0,0,0,34325,22250,0,0,0,0
1,1kmN6466E0369,940625,0,0,0,0,0,0,0,0,20475,38900,0,0,0,0
2,1kmN6467E0369,940600,0,0,0,0,0,0,0,0,37250,22150,0,0,0,0
3,1kmN6468E0369,750700,0,0,0,0,0,0,0,0,219275,30025,0,0,0,0
4,1kmN6465E0370,982525,0,0,0,0,0,0,0,0,8925,8550,0,0,0,0


In [8]:
b=etak_14.columns.tolist()
b.remove('GRD_INSPIR') #Land use cols

In [9]:
etak_14=etak_14.apply(lambda x: (x*0.0001) if x.name in b else x) #get percentages

In [10]:
etak_14

,GRD_INSPIR,BUILT_UP_MIXED_USE,BUILT_UP_COMMERCIAL_USE,OPEN_SPACE_MIXED_USE,WATER,NATURAL_VEGETATION,OTHER,FOREST,CROPLAND,ROADS,WETLANDS,OPEN_SPACE_COMMERCIAL,BUILT_UP_AGRICULTURE,URBAN_GREENERY,AGRICULTURAL_LAND,EXTRACTION_SITES
0,1kmN6465E0369,0.0,0.0,0.0,94.5275,1.7250,3.7475,0.0000,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0
1,1kmN6466E0369,0.0,0.0,0.0,93.4575,2.1400,4.4025,0.0000,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0
2,1kmN6467E0369,0.0,0.0,0.0,92.8975,3.0050,4.0975,0.0000,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0
3,1kmN6468E0369,0.0,0.0,0.0,73.5300,21.5400,4.9300,0.0000,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0
4,1kmN6465E0370,0.0,0.0,0.0,98.2925,1.0375,0.6700,0.0000,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45528,1kmN6547E0707,0.0,0.0,0.0,0.0325,0.0000,11.9500,75.9825,0.0,0.065,11.9700,0.0,0.0,0.0,0.0,0.0
45529,1kmN6548E0707,0.0,0.0,0.0,0.0000,0.0000,0.0000,96.4375,0.0,0.000,3.5625,0.0,0.0,0.0,0.0,0.0
45530,1kmN6549E0707,0.0,0.0,0.0,0.0000,0.0000,0.0000,94.0375,0.0,0.000,5.9625,0.0,0.0,0.0,0.0,0.0
45531,1kmN6550E0707,0.0,0.0,0.0,0.0000,0.0000,0.0000,89.2250,0.0,0.000,10.7750,0.0,0.0,0.0,0.0,0.0


In [11]:
etak_18=etak_18.apply(lambda x: (x*0.0001) if x.name in b else x) #get percentages

In [ ]:
etak_18

In [12]:
from datetime import timedelta, date

#funtion for dates
def daterange (start,end):
    for n in range (int((end-start).days)+1):
        yield start + timedelta(n)
    
hwp_18 = []
hwp_14 =[]
hwp_10 = []


start = date(2018, 7, 10)
end =  date(2018, 8, 13)
for date in daterange (start,end):
    hwp_18.append(date.strftime("%Y-%m-%d"))

   


In [13]:
from datetime import timedelta, date

start = date(2014, 7, 6)
end =  date(2014, 8, 12)
for date in daterange (start,end):
    hwp_14.append(date.strftime("%Y-%m-%d"))

In [14]:
from datetime import timedelta, date
start = date(2010, 7, 3)
end =  date(2010, 8, 18)
for date in daterange (start,end):
    hwp_10.append(date.strftime("%Y-%m-%d"))

In [15]:
#funtction to extract coordantes as tuple
def getXY(pt):
    return (pt.x,pt.y)

#get coordinates from geodataframe
points = grids['geometry'].centroid




#extract coordinate as list of tuples
pointtups = list(map(getXY, points))

grids['centroids']= pointtups


print(len(grids))

grids=grids.drop(grids.index[[28194,28195]]) #remove problematic grids

grids['x']=grids['geometry'].centroid.x
grids['y']=grids['geometry'].centroid.y

print(len(grids))
#len(pointtups)==len(grids) #check

45668
45666


In [16]:
#convert date to DOY
def datestdtojd (stddate):
    fmt='%Y-%m-%d'
    sdtdate = datetime.datetime.strptime(stddate, fmt)
    sdtdate = sdtdate.timetuple()
    doy = sdtdate.tm_yday
    return(doy)

k= datestdtojd(hwp_18[0])
k

191

In [17]:
all_data_18= pd.DataFrame()

for i in range(len(hwp_18)-1):
    
    
    temp= pd.DataFrame()
    
    temp['GRD_INSPIR']=grids['GRD_INSPIR']
    
    lst_path =  os.path.join(lst_18_dir,(hwp_18[i]+image_wc) ).replace('\\','/')
    
    ndvi_path =  os.path.join(ndvi_18_dir,(hwp_18[i]+image_wc) ).replace('\\','/')
    
    
    lst_img = rio.open(lst_path)
    
    ndvi_img = rio.open(ndvi_path)
    
    lst_points  = list(lst_img.sample(grids['centroids']))
    
    ndvi_points  = list(ndvi_img.sample(grids['centroids']))
    
    #flatten the out put to a list
    lst_grid = np.concatenate(lst_points).ravel().tolist()
    
    ndvi_grid = np.concatenate(ndvi_points).ravel().tolist()
    
    temp['ndvi'] = ndvi_grid
    
    temp['lst'] = lst_grid
    
    temp = temp.merge(etak_18,on='GRD_INSPIR')
    
    temp =temp.merge(grid_elev[['GRD_INSPIR','elevation']],on='GRD_INSPIR')
    
    temp =temp.merge(grid_dist[['GRD_INSPIR','dist_coast']],on='GRD_INSPIR')
    
    temp =temp.merge(grids[['GRD_INSPIR','x','y']],on='GRD_INSPIR')
    
    temp['doy'] = datestdtojd(hwp_18[i])
    
    temp['year'] = 2018
    
    all_data_18=pd.concat([all_data_18,temp])
    
    


In [18]:
all_data_14= pd.DataFrame()

for i in range(len(hwp_14)-1):
    
    
    temp= pd.DataFrame()
    
    temp['GRD_INSPIR']=grids['GRD_INSPIR']
    
    lst_path =  os.path.join(lst_14_dir,(hwp_14[i]+image_wc) ).replace('\\','/')
    
    ndvi_path =  os.path.join(ndvi_14_dir,(hwp_14[i]+image_wc) ).replace('\\','/')
    
    
    lst_img = rio.open(lst_path)
    
    ndvi_img = rio.open(ndvi_path)
    
    lst_points  = list(lst_img.sample(grids['centroids']))
    
    ndvi_points  = list(ndvi_img.sample(grids['centroids']))
    
    #flatten the out put to a list
    lst_grid = np.concatenate(lst_points).ravel().tolist()
    
    ndvi_grid = np.concatenate(ndvi_points).ravel().tolist()
    
    temp['ndvi'] = ndvi_grid
    
    temp['lst'] = lst_grid
    
    temp = temp.merge(etak_18,on='GRD_INSPIR')
    
    temp =temp.merge(grid_elev[['GRD_INSPIR','elevation']],on='GRD_INSPIR')
    
    temp =temp.merge(grid_dist[['GRD_INSPIR','dist_coast']],on='GRD_INSPIR')
    
    temp =temp.merge(grids[['GRD_INSPIR','x','y']],on='GRD_INSPIR')
    
    temp['doy'] = datestdtojd(hwp_14[i])

    temp['year'] = 2014
    
    all_data_14=pd.concat([all_data_14,temp])
    

In [20]:
all_data_10= pd.DataFrame()

for i in range(len(hwp_10)-1):
    
    
    temp= pd.DataFrame()
    
    temp['GRD_INSPIR']=grids['GRD_INSPIR']
    
    lst_path =  os.path.join(lst_10_dir,(hwp_10[i]+image_wc) ).replace('\\','/')
    
    ndvi_path =  os.path.join(ndvi_10_dir,(hwp_10[i]+image_wc) ).replace('\\','/')
    
    
    lst_img = rio.open(lst_path)
    
    ndvi_img = rio.open(ndvi_path)
    
    lst_points  = list(lst_img.sample(grids['centroids']))
    
    ndvi_points  = list(ndvi_img.sample(grids['centroids']))
    
    #flatten the out put to a list
    lst_grid = np.concatenate(lst_points).ravel().tolist()
    
    ndvi_grid = np.concatenate(ndvi_points).ravel().tolist()
    
    temp['ndvi'] = ndvi_grid
    
    temp['lst'] = lst_grid
    
    temp = temp.merge(etak_18,on='GRD_INSPIR')
    
    temp =temp.merge(grid_elev[['GRD_INSPIR','elevation']],on='GRD_INSPIR')
    
    temp =temp.merge(grid_dist[['GRD_INSPIR','dist_coast']],on='GRD_INSPIR')
    
    temp =temp.merge(grids[['GRD_INSPIR','x','y']],on='GRD_INSPIR')

    temp['doy'] = datestdtojd(hwp_10[i])
    
    temp['year'] = 2010
    
    all_data_10=pd.concat([all_data_10,temp])
    

In [21]:
all_data_18.head()

,GRD_INSPIR,ndvi,lst,WATER,ROADS,URBAN_GREENERY,OTHER,BUILT_UP_COMMERCIAL_USE,EXTRACTION_SITES,OPEN_SPACE_COMMERCIAL,...,FOREST,WETLANDS,BUILT_UP_MIXED_USE,BUILT_UP_AGRICULTURE,elevation,dist_coast,x,y,doy,year
0,1kmN6465E0369,0.109375,NaN,94.3425,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.127982,0.0,369500.0,6465500.0,191,2018
1,1kmN6466E0369,0.109375,NaN,94.0625,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.729734,0.0,369500.0,6466500.0,191,2018
2,1kmN6467E0369,0.109375,NaN,94.0600,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.869551,0.0,369500.0,6467500.0,191,2018
3,1kmN6468E0369,0.109375,NaN,75.0700,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.869971,0.0,369500.0,6468500.0,191,2018
4,1kmN6465E0370,0.109375,NaN,98.2525,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.171496,0.0,370500.0,6465500.0,191,2018


In [22]:
all_data_14.head()

,GRD_INSPIR,ndvi,lst,WATER,ROADS,URBAN_GREENERY,OTHER,BUILT_UP_COMMERCIAL_USE,EXTRACTION_SITES,OPEN_SPACE_COMMERCIAL,...,FOREST,WETLANDS,BUILT_UP_MIXED_USE,BUILT_UP_AGRICULTURE,elevation,dist_coast,x,y,doy,year
0,1kmN6465E0369,0.100402,NaN,94.3425,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.127982,0.0,369500.0,6465500.0,187,2014
1,1kmN6466E0369,0.100402,NaN,94.0625,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.729734,0.0,369500.0,6466500.0,187,2014
2,1kmN6467E0369,0.100402,NaN,94.0600,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.869551,0.0,369500.0,6467500.0,187,2014
3,1kmN6468E0369,0.100402,NaN,75.0700,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.869971,0.0,369500.0,6468500.0,187,2014
4,1kmN6465E0370,0.100402,NaN,98.2525,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.171496,0.0,370500.0,6465500.0,187,2014


In [23]:
all_data_10.head()

,GRD_INSPIR,ndvi,lst,WATER,ROADS,URBAN_GREENERY,OTHER,BUILT_UP_COMMERCIAL_USE,EXTRACTION_SITES,OPEN_SPACE_COMMERCIAL,...,FOREST,WETLANDS,BUILT_UP_MIXED_USE,BUILT_UP_AGRICULTURE,elevation,dist_coast,x,y,doy,year
0,1kmN6465E0369,0.238411,NaN,94.3425,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.127982,0.0,369500.0,6465500.0,184,2010
1,1kmN6466E0369,0.238411,NaN,94.0625,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.729734,0.0,369500.0,6466500.0,184,2010
2,1kmN6467E0369,0.238411,NaN,94.0600,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.869551,0.0,369500.0,6467500.0,184,2010
3,1kmN6468E0369,0.238411,NaN,75.0700,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.869971,0.0,369500.0,6468500.0,184,2010
4,1kmN6465E0370,0.238411,NaN,98.2525,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.171496,0.0,370500.0,6465500.0,184,2010


In [24]:
test=all_data_10[all_data_10['ndvi'].notnull() & all_data_10['lst'].notnull()]
test#[test['doy']==184]

,GRD_INSPIR,ndvi,lst,WATER,ROADS,URBAN_GREENERY,OTHER,BUILT_UP_COMMERCIAL_USE,EXTRACTION_SITES,OPEN_SPACE_COMMERCIAL,...,FOREST,WETLANDS,BUILT_UP_MIXED_USE,BUILT_UP_AGRICULTURE,elevation,dist_coast,x,y,doy,year
15,1kmN6464E0372,0.806931,20.27,82.0000,0.0000,0.0,1.3400,0.0,0.0,0.0000,...,3.5225,0.0000,0.0000,0.0,1.597276,0.000000,372500.0,6464500.0,184,2010
22,1kmN6461E0373,0.800000,21.17,85.0650,0.0000,0.0,0.9125,0.0,0.0,0.0000,...,1.0100,0.2225,0.0000,0.0,0.835327,0.000000,373500.0,6461500.0,184,2010
24,1kmN6463E0373,0.853890,19.77,36.6000,0.0075,0.0,4.7450,0.0,0.0,0.0000,...,41.6025,1.3725,0.0000,0.0,4.136928,0.000000,373500.0,6463500.0,184,2010
25,1kmN6464E0373,0.855664,21.96,0.0000,0.0725,0.0,8.5475,0.0,0.0,0.0000,...,55.5075,1.9950,0.0200,0.0,4.517952,51.769712,373500.0,6464500.0,184,2010
31,1kmN6485E0545,0.867167,25.20,0.0875,0.0225,0.0,2.6575,0.0,0.0,0.0000,...,91.8600,0.0000,0.0575,0.0,30.977317,20828.047540,545500.0,6485500.0,184,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45634,1kmN6446E0707,0.706752,22.43,10.9000,0.0000,0.0,0.9650,0.0,0.0,0.0000,...,0.6575,5.3725,0.0175,0.0,30.905664,143465.658300,707500.0,6446500.0,229,2010
45635,1kmN6447E0707,0.742213,21.98,5.8000,0.0000,0.0,1.2600,0.0,0.0,0.0000,...,0.7400,0.0000,0.0000,0.0,30.587915,142467.245300,707500.0,6447500.0,229,2010
45636,1kmN6448E0707,0.767683,22.45,30.3675,0.7700,0.0,0.2725,0.0,0.0,0.1325,...,19.9075,3.1800,0.1575,0.0,30.813690,141468.854800,707500.0,6448500.0,229,2010
45637,1kmN6449E0707,0.620553,21.64,65.9400,0.0000,0.0,0.2525,0.0,0.0,0.0000,...,3.1450,9.4450,0.0000,0.0,30.417095,140470.487100,707500.0,6449500.0,229,2010


In [25]:
test=all_data_10[(all_data_10['elevation']==-9999.000000) & all_data_10['lst'].isnull()]
test#[test['doy']==188]

,GRD_INSPIR,ndvi,lst,WATER,ROADS,URBAN_GREENERY,OTHER,BUILT_UP_COMMERCIAL_USE,EXTRACTION_SITES,OPEN_SPACE_COMMERCIAL,...,FOREST,WETLANDS,BUILT_UP_MIXED_USE,BUILT_UP_AGRICULTURE,elevation,dist_coast,x,y,doy,year


In [26]:
all_data = pd.concat([all_data_10,all_data_14,all_data_18])

In [ ]:
all_data

In [27]:
def built_up (row):
    bs = row ['BUILT_UP_COMMERCIAL_USE'] + row['BUILT_UP_MIXED_USE'] + row ['BUILT_UP_AGRICULTURE'] 
    return bs


def building_surface (row):
    bs = row ['BUILT_UP_COMMERCIAL_USE'] + row['BUILT_UP_MIXED_USE'] + row ['BUILT_UP_AGRICULTURE'] 
    return bs

def imperv_surface(row):
    ips = row ['ROADS'] + row['OPEN_SPACE_COMMERCIAL'] + row ['OPEN_SPACE_MIXED_USE']
    return ips 

def perv_surface(row):
    ps = row ['WATER'] + row['URBAN_GREENERY'] + row ['OTHER']+ row['EXTRACTION_SITES'] + row ['CROPLAND']+ row ['AGRICULTURAL_LAND']+ row ['FOREST']+ row ['WETLANDS']+ row ['NATURAL_VEGETATION']
    return ps 

In [28]:
all_data['building surface'] = all_data.apply(building_surface, axis=1)



all_data['impervious surface'] = all_data.apply(imperv_surface, axis=1)



all_data['pervious surface'] = all_data.apply(perv_surface, axis=1)

all_data['BUILT_UP'] = all_data.apply(built_up, axis=1)


In [32]:
all_data.to_csv(Out_folder+'all_data_final.csv',index=False)

In [33]:
#seperate data for learning and future predictions

##records with number values for lst and ndvi are suitable for learning

all_data_learn = all_data[all_data['ndvi'].notnull() & all_data['lst'].notnull()]

##records with NAN lst values but with number values for ndvi will be predicted with the final model

all_to_be_pred_data = all_data[all_data['ndvi'].notnull() & all_data['lst'].isnull()]

##records with number values for lst values and NAN values fr ndvi will be kept 
add_data = all_data[all_data['lst'].notnull() & all_data['ndvi'].isnull()]

##records with NAN for both lst and ndvi will be eliminated.

In [37]:
data_10 = all_data_learn[all_data_learn['year']==2010]
print(data_10.shape)

data_14 = all_data_learn[all_data_learn['year']==2014]
print(data_14.shape)

data_18 = all_data_learn[all_data_learn['year']==2018]
print(data_18.shape)

(1367678, 28)
(1240437, 28)
(1003579, 28)


In [39]:
data_10_s = data_10.sample(frac=0.40,random_state=42)
print(data_10_s.shape)
data_14_s = data_14.sample(frac=0.44,random_state=42)
print(data_14_s.shape)
data_18_s = data_18.sample(frac=0.54,random_state=42)
print(data_18_s.shape)

(547071, 28)
(545792, 28)
(541933, 28)


In [42]:
all_to_be_pred_data.to_csv(Out_folder+'to_pred_final.csv',index=False)

pd.concat([data_10_s,data_14_s,data_18_s]).to_csv(Out_folder+'learn_data_final.csv',index=False)

add_data.to_csv(Out_folder+'extra_data_final.csv',index=False)